# Read a file from HDFS and count the words #
This is a simple PySpark program which reads a file from a remote Hadoop filesystem and counts the words it contains. To run this example you need a spark cluster running in your project, and you need to know the host and port for a reachable Namenode in an external Hadoop cluster.

Note, this example will run as the user *nbuser*, so the HDFS filesystem path needs to be readable by *nbuser* on the Hadoop cluster.

## Connect to the Spark cluster ##

This example assumes that a Spark cluster is already running in your project. Change *spark://mycluster:7077* to the URL of the Spark master in the block below. Running this block creates the SparkContext, so you only want to run it once.

In [1]:
import pyspark
from pyspark.context import SparkContext
spc = SparkContext("spark://mycluster:7077")

## Set the HDFS host, port, and input path ##
The *hdfs_host* value should be the hostname of the machine running your hadoop Namenode, and the *hdfs_port* should be the port that the Namenode is listening on.  The default port is 8020. If a different port has been set, it can be found in the *fs.defaultFS* property in the *core-site.xml* configuration file on your Hadoop cluster.

The *hdfs* path should be the absolute path of the file that you want to read from HDFS.

In [2]:
hdfs_host = "myhost.me.com"
hdfs_port = 8020
hdfs_path = "/user/me/input"

## Read the file and print the counts for  up to 20 words ##
This block reads the file and splits up words on the space character. It turns the resulting RDD into a list using *collect()* and then prints up to 20 key/value pairs showing the key and the number of occurrences for each.

The second line in this block is the most important -- this is where Spark actually reads the file from HDFS.

In [ ]:
import os
text_file = spc.textFile("hdfs://%s:%d%s" % (hdfs_host, hdfs_port, os.path.join("/", hdfs_path)))
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
values = counts.collect()
if len(values) > 20:
    values = values[:20]
print("\n".join([x[0] + " = " + str(x[1]) for x in values]))